# VCノードの作成--IdP-proxy

---

[IdP-proxy](https://github.com/NII-cloud-operation/CoursewareHub-LC_idp-proxy)を構築するためノードをVCP SDKを用いて作成します。

## はじめに


![構成](images/cw-221-01.png)

このNotebookでは上図にある IdP-proxyコンテナを実行するためのVCノードを作成します。

### 事前に用意が必要となるものについて

このNotebookを実行するにあたって、あらかじめ準備が必要となるものについて以下に記します。

* VCCアクセストークン
* SSH公開鍵ペア
* VCノードに割り当てるアドレス
* NTPの設定

####  VCCアクセストークン

このNotebookではVCP SDKを用いてVCノード(クラウドの計算資源)を作成します。VCP SDKを用いてVC Controller(VCC)の操作を行うために VCCアクセストークンが必要となります。

VCCアクセストークンがない場合はVC管理者にアクセストークンの発行を依頼してください。

#### SSH公開鍵ペア

作成するVCノードに登録するSSHの公開鍵を用意する必要があります。あらかじめ作成したものをこのNotebook環境内にアップロードするか、この環境内で公開鍵ペアの作成を行ってください。

#### VCノードに割り当てるアドレス

VCノードのネットワークインターフェースに割り当てるアドレスを用意してください。VCノードにはIPアドレス、またはMACアドレス(VMwareのみ)を割り当てることができます。

#### NTPの設定

IdPと連携して認証を行う場合は時刻が正しく設定されていないと正しく動作しないことがあります。時刻合わせを行うためにVCノードのNTPを設定するには事前にVCコントローラへの設定が必要となります。
VCPのリリースノート「[Release/20.10.0 -- 2.機能追加](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=32677360#id-%E3%83%AA%E3%83%AA%E3%83%BC%E3%82%B9%E3%83%8E%E3%83%BC%E3%83%88-Release/20.10.0(2020/10/30))」に記されているように、OCS運用担当者にVCコントローラへのNTP設定を依頼してください。

## VCノードに関するパラメータ

IdP-Proxyの構築環境となるVCノードに関するパラメータを指定します。

### VCCアクセストークンの入力

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
ERROR - config vc failed: http_status(403)
ERROR - 2023/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### UnitGroup名の指定

VCPの構成要素は以下のようになっています。

* VCノード
  - クラウドにおける計算資源(VM/BM)
  - 例えば Amazon EC2インスタンス, Microsoft Azure VM など
* Unit
  - 同質のVCノードにより構成されている要素
  - 同じUnitに属するVCノードはCPU,メモリ等の計算資源が全て同じ設定になっている
* UnitGroup
  - 複数のUnitにより構成されている要素
  - 使用目的、ライフサイクルなどに合わせて、複数のUnitをまとめて扱うための要素

UnitGroupの名前を指定します。既存のUnitGroup名を確認するために現在のUnitGroup一覧を表示します。次のセルを実行し`vcname`に表示されている名前が既存のUnitGroup名となります。

In [ ]:
vcp.df_ugroups()

このNotebookで作成するUnitGroup名を指定してください。上記の一覧に表示されている名前との重複を避けて下さい。またIdP-proxyのUnitGroup名には英数字のみの値を指定してください（先頭文字に数字は指定できない）。

In [ ]:
# (例)
# ugroup_name = 'IdPproxy'

ugroup_name = 

### クラウドプロバイダの指定

VCノードを起動するプロバイダを選択します。

In [ ]:
# (例)
# vc_provider = 'aws'
# vc_provider = 'azure'

vc_provider = 

### VCノードに割り当てるリソース量の指定

VCノードに割り当てるリソース量を指定します。

ここでは、事前にVCP SDKで設定している `flavor` を指定することで IdP proxyに割り当てるリソース量を指定します。次のセルを実行すると `vc_provider` に設定したプロバイダに対応する `flavor` の一覧が表示されます。

In [ ]:
vcp.df_flavors(vc_provider)

表示された `flavor` の値から一つを選択して、次のセルに指定してください。

In [ ]:
# (例)
# idp_proxy_flavor = 'small'

idp_proxy_flavor = 

### アドレスの指定

VCノードのネットワークインターフェースに割り当てるアドレスを指定します。

#### IPアドレスを指定する場合

VCノードにIPアドレスを割り当てる場合はこの節を実行してください。
IPアドレスを指定しない場合はVCノードに割り当て可能なアドレスが自動的に設定されます

まず指定可能なIPアドレスの範囲を確認します。次のセルを実行するとVCノードに割り当て可能なIPアドレスの範囲が表示されます。

In [ ]:
print(vcp.get_vpn_catalog(vc_provider).get('private_network_ipmask'))

次のセルで作成するVCノードに割り当てるプライベートネットワークのIPアドレスを指定してください。

In [ ]:
# (例)
# vc_ipaddress = '172.30.2.100'

vc_ipaddress = 

#### MACアドレスを指定する場合

VCノードにMACアドレスを割り当てる場合はこの節を実行してください。

> MACアドレス指定が可能なのは `vc_provider` に `vmware` を指定した場合に限られます。

In [ ]:
# (例)
# vc_mac_address = '4a:d1:4d:ab:cf:12'

vc_mac_address = 

### SSH公開鍵認証の鍵ファイルの指定

起動したVCノードにsshでログインして操作を行うためにSSH公開鍵を登録します。

VCノードに登録するSSHの公開鍵ファイルのパスを次のセルで指定してください。

> 事前に作成した公開鍵をこの環境にアップロードするか、この環境内でSSH公開鍵の作成を行ってください。

In [ ]:
# (例)
# ssh_public_key_path = '~/.ssh/id_rsa.pub'

ssh_public_key_path = 

公開鍵に対応する秘密鍵のパスを次のセルで指定してください。

In [ ]:
# (例)
# ssh_private_key_path = '~/.ssh/id_rsa'

ssh_private_key_path = 

### パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

後の手順でVCノードに対する操作を、構成管理ツールの[Ansible](https://www.ansible.com/)で行います。そこで、パラメータの保存形式は `Ansible` のフォーマットに従うことにします。Ansible では `group_vars/`というディレクトリに YAML フォーマットのファイルを配置すると、そのファイルに記録されている値を変数として利用することができます。このNotebookではパラメータを `group_vars/` のYAMLファイルとして保存します。

ファイルに保存する前にパラメータに関する簡易なチェックを行います。エラーになった場合はその後に表示される指示に従ってください。

In [ ]:
%run scripts/check_params.py
check_parameters(
    'ugroup_name',
    'vc_provider',
    'idp_proxy_flavor',
    'vc_ipaddress',
    'vc_mac_address',
    'ssh_public_key_path',
    'ssh_private_key_path',
    params={
        'vcp': vcp,
        'opt_vars': ['vc_ipaddress', 'vc_mac_address'],
    },
    nb_vars=locals(),
)

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

In [ ]:
import os
%run scripts/group.py
update_group_vars(
    ugroup_name,
    ugroup_name=ugroup_name,
    vc_provider=vc_provider,
    idp_proxy_flavor=idp_proxy_flavor,
    ssh_public_key_path=os.path.expanduser(ssh_public_key_path),
    ssh_private_key_path=os.path.expanduser(ssh_private_key_path),
)
if 'vc_ipaddress' in vars():
    update_group_vars(ugroup_name, vc_ipaddress=vc_ipaddress)
if 'vc_mac_address' in vars():
    update_group_vars(ugroup_name, vc_mac_address=vc_mac_address)

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

In [ ]:
!cat group_vars/{ugroup_name}

## VCノードの起動

VCノードを起動します。

`spec` を指定します。

In [ ]:
import sys
%run scripts/group.py
gvars = load_group_vars(ugroup_name)
spec = vcp.get_spec(gvars['vc_provider'], gvars['idp_proxy_flavor'])
spec.image = 'harbor.vcloud.nii.ac.jp/vcp/base:1.8.1-centos7-x86_64-rc1'

if 'vc_ipaddress' in gvars:
    # VCノードに割り当てるIPアドレスを指定する
    spec.ip_addresses = [gvars['vc_ipaddress']]
if 'vc_mac_addresses' in gvars:
    # VCノードに割り当てるMACアドレスを指定する
    spec.mac_addresses = [gvars['vc_mac_address']]
    
# ルートボリュームサイズを指定する
if 'idp_proxy_disk_size' in gvars:
    if vc_provider == 'aws':
        spec.volume_size = gvars['idp_proxy_disk_size']
    elif vc_provider == 'azure':
        spec.disk_size_gb = gvars['idp_proxy_disk_size']
    elif vc_provider == 'oracle':
        spec.boot_volume_size_in_gbs = gvars['idp_proxy_disk_size']
    else:
        print('This provider does not support the specification of the root volume size. Does not set the disk size.', file=sys.stderr)

# VCノードにsshでログインするための公開鍵を指定する
spec.set_ssh_pubkey(gvars['ssh_public_key_path'])

`spec` の設定値を確認します。

In [ ]:
print(spec)

UnitGroupを作成します。

In [ ]:
ugroup = vcp.create_ugroup(ugroup_name)

IdP-proxy用VCノードを起動します。

In [ ]:
unit = ugroup.create_unit('idpproxy', spec)

起動したVCノードの一覧を表示します。

In [ ]:
unit.df_nodes()

VCノードのIPアドレスを変数`vc_ipaddress`に設定します。

In [ ]:
vc_ipaddress = unit.find_ip_addresses()[0]
print(vc_ipaddress)

`group_vars`ファイルにIPアドレスの値を記録します。

In [ ]:
update_group_vars(ugroup_name, vc_ipaddress=vc_ipaddress)
gvars = load_group_vars(ugroup_name)

## Ansibleの設定

VCノードをAnsibleで操作するための設定を行います。

まず、VCノードにSSHでログインできるようにするために `~/.ssh/known_hosts` の更新を行います。

> 何度かVCノードの起動を行うと、異なるホストが同じIPアドレスで起動するためにSSHのホストキーのチェックでエラーになる事があります。このような状況に対応するために、起動したVCノードのIPアドレスに対応するエントリを`known_hosts`ファイルから削除します。その後、`ssh-keyscan`コマンドを利用して起動したVCノードのホストキーを取得して `known_hosts`ファイルの内容を更新します。

In [ ]:
from time import sleep

def check_update_known_hosts(ipaddr):
    # VCノード起動直後だと sshd サービスが開始されておらずに known_hosts が更新されない場合がある
    # ssh-keyscan が値を取得できるまで何度かリトライする
    for x in range(10):
        out = ! echo $(ssh-keyscan {ipaddr} 2> /dev/null | wc -l)
        update_lines = int(out[0])
        if update_lines > 0:
            break
        sleep(1)
    else:
        raise RuntimeError("ERROR: timeout!")    

!mkdir -p -m 0700 ~/.ssh
!touch ~/.ssh/known_hosts
for addr in ugroup.find_ip_addresses():
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

起動したVCノードに対応するエントリを Ansible のインベントリに登録します。

> Ansibleで操作を行うためには、操作対象のホスト(IPアドレス)をインベントリに登録する必要があります。

In [ ]:
%run scripts/group.py
%run scripts/edit_conf.py

inventory = {'all': {'children': {
    ugroup.name: {
        'children': {
            f'{ugroup.name}_{unit.name}': {
                'hosts': dict([(x, dict(servicenet_ip=x)) for x in unit.find_ip_addresses()]),
            },
        },
        'vars': {
            'ansible_user': 'root',
            'ansible_ssh_private_key_file': gvars['ssh_private_key_path'],
            'ansible_python_interpreter': '/usr/bin/python3',
        },
    },
}}}

generate_edit_link(update_inventory_yml(inventory))

次のセルを実行すると作成したインベントリの内容を表示します。インベントリの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat inventory.yml

 先程VCノードを登録したファイルをインベントリとして指定するためのAnsibleのコンフィギュレーションファイルを作成します。
> カレントディレクトリにコンフィギュレーションファイル(`ansible.cfg`)を作成すると、Ansibleを実行する際にその設定が適用されます。

In [ ]:
cfg = setup_ansible_cfg()
generate_edit_link(cfg)

次のセルを実行すると作成したコンフィギュレーションファイルの内容を表示します。コンフィギュレーションファイルの内容を変更したい場合は、上のセルの出力結果に表示しているリンクから編集することができます。

In [ ]:
!cat ansible.cfg

UnitGroupに属する全てのVCノードに対して Ansible で接続できることを確認します。

> ここでは、複数のVCノードをまとめて扱うためにAnsibleのグループを指定しています。グループ名は UnitGroup名と同じ値にしてあります。

In [ ]:
!ansible {ugroup.name} -m ping

必要となるパッケージのインストールなどを行います。

In [ ]:
!ansible-playbook -l {ugroup.name} -Dv playbooks/install-bc-additional-packages.yml